In [1]:
#download catalogue from INGV

# %% LIB
from obspy.clients.fdsn.client import Client
from obspy import UTCDateTime
from obspy.core.event import Catalog
from obspy.core.stream import Stream
from obspy.core.event import Event
from obspy.core.event import Origin
from obspy.core.event import Magnitude
from obspy import read_events
from obspy import read_inventory
from obspy.core.event import ResourceIdentifier
import cartopy.crs as ccrs

from pyrocko import util, model, io, trace, moment_tensor, gmtpy
from pyrocko import pz
from pyrocko import orthodrome as od
from pyrocko.io import quakeml
from pyrocko.io import stationxml as fdsn
from pyrocko.client import catalog
from pyrocko.automap import Map

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import csv

In [2]:
workdir= '../'
catdir =  os.path.join(workdir,'CAT')

client=Client('INGV')

stime=UTCDateTime('2014-01-01T00:00:00')        # CHANGE set start time
etime=UTCDateTime('2024-11-01T00:00:00')        # CHANGE set end time

minlat=31                                       # CHANGE
maxlat=32                                        # CHANGE
minlon=131                                        # CHANGE
maxlon=132                                      # CHANGE

minmag=7.

catalogue_filename= 'cat_eq_japan'              # CHANGE

print('downloading events from INGV database')
cat_INGV=client.get_events(starttime=stime,endtime=etime,
                        minlatitude=minlat,maxlatitude=maxlat,minlongitude=minlon,
                        maxlongitude=maxlon,minmagnitude=minmag)


cat_name=os.path.join(catdir,catalogue_filename + '.xml')
cat_INGV.write(cat_name,format='QUAKEML')

print('number of events selected:',len(cat_INGV))
print(cat_INGV)

downloading events from INGV database
number of events selected: 1
1 Event(s) in Catalog:
2024-08-08T07:42:54.173000Z | +31.690, +131.551 | 7.1  Mwp | automatic


In [3]:
# create/load pf catalogue
catname_pf = os.path.join(catdir, catalogue_filename + '.pf')

def catalogue_INGV_to_PF(cat,cat_name):
    events = []
    for ev in cat:
        name= str(ev.resource_id).split('=')[1]

        timetmp= str(ev.origins[0].time)
        timestr = timetmp[0:10] + ' ' + timetmp[11:23]                          # format: '2010-01-01 00:00:00.000'
        time = util.str_to_time(timestr)

        lat=    float( ev.origins[0].latitude )
        lon =   float( ev.origins[0].longitude )
        depth = float( ev.origins[0].depth )
        magnitude = float( ev.magnitudes[0].mag )
        events.append(model.Event(name=name, time=time,
                                lat=lat, lon=lon,
                                depth=depth, magnitude=magnitude))
    events.sort(key=lambda x: x.time, reverse=False)
    model.dump_events(events, cat_name)
    return events

print('creating catalogues INGV in .pf')
cat_pf_INGV=    catalogue_INGV_to_PF(cat_INGV,catname_pf)

creating catalogues INGV in .pf


In [ ]:
# %% EXTRA: create .txt file of the catalogue

# columns:  TIME    LAT     LON     DEPTH   MAGNITUDE
######################################################################################
######################################################################################
switch_create_txt_file_with_catalogue= True      ########################SWITCH
######################################################################################
######################################################################################
def create_txt_catalogue(catalogue):
    list_events=[]
    for ev in catalogue:
        list_events.append( util.time_to_str(ev.time)+'\t'+str(ev.lat)
                           +'\t'+str(ev.lon)+'\t' +str(ev.depth)+'\t'+str(ev.magnitude) )
    return list_events

if switch_create_txt_file_with_catalogue:
    print('Creating a txt file of the selected catalogue')
    cat=events                                                          #CHANGE
    l_ev=create_txt_catalogue(cat)
    catname = os.path.join(catdir, 'catalogue_flegrei_mag_2_5.txt')     #CHANGE

    with open(catname, 'w') as output:
        for row in l_ev:
            output.write(str(row) + '\n')


# %% CHECK DATA NAME'S CONSISTENCIES:
# DATA
# BLACKLIST
# MARKERS